In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from sklearn.decomposition import PCA

Melakukan read rawdata dan dibagi menjadi x(feature) dan y(target). 
Setelah itu train test split dilakukan

In [ ]:
data = pd.read_csv('rawdata.csv')

data = data.drop("SAMPLE_ID", axis = 1)
data["STATUS"].replace({1:0,2:1}, inplace=True)

x = data.drop("STATUS", axis=1)
y = data['STATUS']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

Sebelum mereshape, x_train dan x_test diubah dalam bentuk array

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

PCA data x_train dan x_test

In [ ]:
pca = PCA(n_components=64)
pca = pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

Kemudian reshape x_train dan x_test sesuai dengan shape nya

In [ ]:
x_train = x_train.reshape(71, 64, 1)
x_test = x_test.reshape(18, 64, 1)

Membuat model dan menambahkan layer dari model

In [ ]:
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(64,1)))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

Melakukan compile model untuk mengetahui measure model performance

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 62, 64)            256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 60, 32)            6176      
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1921      
Total params: 8,353
Trainable params: 8,353
Non-trainable params: 0
_________________________________________________________________


Validation

In [ ]:
x_test, x_validation, y_test, y_validation = train_test_split(x_test, y_test, test_size = 0.3)

Training model

In [ ]:
model.fit(x_train, y_train, validation_data=(x_validation, y_validation), epochs=10)

Epoch 1/10
3/3 [==============================] - 16s 285ms/step - loss: 0.6769 - accuracy: 0.5449 - val_loss: 0.7225 - val_accuracy: 0.1667
Epoch 2/10
3/3 [==============================] - 0s 16ms/step - loss: 0.6039 - accuracy: 0.6818 - val_loss: 0.7194 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 0s 16ms/step - loss: 0.5802 - accuracy: 0.6920 - val_loss: 0.7159 - val_accuracy: 0.3333
Epoch 4/10
3/3 [==============================] - 0s 15ms/step - loss: 0.5077 - accuracy: 0.8436 - val_loss: 0.7129 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 0s 15ms/step - loss: 0.4829 - accuracy: 0.9250 - val_loss: 0.7100 - val_accuracy: 0.5000
Epoch 6/10
3/3 [==============================] - 0s 15ms/step - loss: 0.4431 - accuracy: 0.8937 - val_loss: 0.7064 - val_accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 0s 16ms/step - loss: 0.4173 - accuracy: 0.8515 - val_loss: 0.7026 - val_accuracy: 0.5000
Epoch 8/10
3/3 [=========

Prediksi test set dan mengubah setiap value > 0.50 menjadi 1 dan setiap value < 0.50 menjadi 0 agar accuracy score dapat dilakukan 

In [ ]:
prediction = model.predict(x_test)
pred = (prediction > 0.50).astype(int)
pred

array([[0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1]])

Hasil akhir dalam bentuk perbandingan antara data prediksi dan data real

In [ ]:
accuracy_score(y_test, pred)

0.4166666666666667